In [1]:
import re

In [2]:
import pandas as pd

In [3]:
import spacy

/home/ubuntu/.conda/envs/jai_new_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Import and process annotation session data

In [4]:
data_session_1=pd.read_json('../../training/data/prodigy_sessions/el_session_1.jsonl',lines=True)
data_session_2=pd.read_json('../../training/data/prodigy_sessions/el_session_2.jsonl',lines=True)

In [5]:
df=data_session_1.append(data_session_2).reset_index()
del(data_session_1)
del(data_session_2)

/tmp/ipykernel_5040/2485285366.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=data_session_1.append(data_session_2).reset_index()


In [6]:
df.shape

(4470, 18)

In [7]:
# Remove instances wrongly accepted without any selected options
df['accept_len']=df['accept'].apply(lambda x: len(''.join(x))).sort_values()
df=df[df['accept_len']!=0]
df.drop('accept_len',1,inplace=True)

/tmp/ipykernel_5040/2620138047.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop('accept_len',1,inplace=True)


In [8]:
df.shape

(4466, 18)

In [9]:
df.loc[:,'ents']=df.loc[:,'spans'].apply(lambda x: x[0]['text'])

In [10]:
df['ents'].nunique()

2057

In [11]:
## Remove entities that show up too often (> 3 times)

In [12]:
ent_counts=pd.DataFrame(df['ents'].value_counts())

In [13]:
ent_counts=ent_counts.reset_index().rename(columns={'index':'ents','ents':'counts'})

In [14]:
punctuation_to_filter=ent_counts.loc[ent_counts['ents'].str.len()==1,'ents'].unique()

In [15]:
punctuation_to_filter

array(['–', '“', '”', '‘', '’'], dtype=object)

In [16]:
ent_counts=ent_counts[~ent_counts['ents'].isin(punctuation_to_filter)]

In [17]:
unwanted_ents=set(ent_counts.loc[ent_counts['counts']>4,'ents'])

In [18]:
data=pd.read_csv('gu_resampled_by_section_id_full_name_all_aliases_kb_match_paragraphs.csv', index_col=0)


In [19]:
data.shape

(152616, 5)

In [20]:
for punctuation_mark in punctuation_to_filter:
    data['paragraphs']=data['paragraphs'].str.replace(punctuation_mark, ' ')

In [21]:
paragraphs=df[['text','_input_hash']]

In [22]:
data.shape

(152616, 5)

In [23]:
paragraphs.shape

(4466, 2)

In [24]:
data=data.drop_duplicates('paragraphs')

In [25]:
paragraphs=paragraphs.drop_duplicates('text')

In [26]:
data.shape

(152084, 5)

In [27]:
paragraphs.shape

(1917, 2)

In [28]:
data['paragraph_match']=data['paragraphs'].apply(lambda x: re.sub(r'\W+', '', x))

In [29]:
paragraphs['paragraph_match']=paragraphs['text'].apply(lambda x: re.sub(r'\W+', '', x))

In [30]:
data=data.merge(paragraphs, how='left', on=['paragraph_match'])

In [31]:
data.drop('paragraph_match',1,inplace=True)

/tmp/ipykernel_5040/3336296306.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop('paragraph_match',1,inplace=True)


In [32]:
data['paragraphs'].isna().any()

False

In [33]:
data=data[data['_input_hash'].isna()]

In [34]:
data.drop(['text','_input_hash'],1,inplace=True)

/tmp/ipykernel_5040/1640300887.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop(['text','_input_hash'],1,inplace=True)


In [35]:
data=data.drop_duplicates(['paragraphs'])

In [36]:
data.shape

(150325, 5)

In [37]:
for ent in unwanted_ents:
    data=data[~data['paragraphs'].str.contains(ent)]

In [38]:
data.shape

(79002, 5)

In [63]:
data.loc[data['alias'].isin(['ed case'])]

,url,paragraphs,doc_index,lower_paragraphs,alias
28,www.theguardian.com/global-development/2018/ma...,"Since January 2017, a measles outbreak has led...",17440,"since january 2017, a measles outbreak has led...",ed case
128,www.theguardian.com/uk-news/2014/oct/06/northu...,This will include contacting the victims in t...,16275,“this will include contacting the victims in t...,ed case
338,www.theguardian.com/world/2017/oct/06/japanese...,It said 368 people 352 of them men had kil...,6826,it said 368 people – 352 of them men – had kil...,ed case
385,www.theguardian.com/world/2020/mar/16/coronavi...,We have a simple message for all countries Te...,5719,“we have a simple message for all countries te...,ed case
598,www.theguardian.com/business/2015/jun/10/obscu...,"A year after the Moorburg case closed, Vattenf...",18151,"a year after the moorburg case closed, vattenf...",ed case
...,...,...,...,...,...
151410,www.theguardian.com/global-development/2020/au...,Four years after the last recorded cases of wi...,17518,four years after the last recorded cases of wi...,ed case
151465,www.theguardian.com/us-news/2022/mar/22/cowboy...,The ruling bolsters a key theory from prosecut...,5094,the ruling bolsters a key theory from prosecut...,ed case
151473,www.theguardian.com/us-news/2020/oct/19/scott-...,"Coronavirus is back on the ascendant, across t...",10381,"coronavirus is back on the ascendant, across t...",ed case
151583,www.theguardian.com/world/2014/oct/17/ebola-ep...,"While the outbreak is now officially over, Se...",13203,"“while the outbreak is now officially over, se...",ed case


In [ ]:
['anita','innocent', 'al from', 'barba', 'ed case', 'christo']

In [74]:
alias_counts=data['alias'].value_counts().reset_index().rename(columns={'index':'alias','alias':'counts'})

In [83]:
less_common_alias=alias_counts.loc[alias_counts['counts']<100,'alias'].unique()

In [88]:
data=data[data['alias'].isin(less_common_alias)]

In [93]:
data=data.groupby(['alias']).sample(3, replace=True, random_state=42)

In [95]:
data=data.drop_duplicates()

In [100]:
data['alias'].value_counts()

michel kafando       3
nick xenophon        3
john randall         3
carly fiorina        3
carme forcadell      3
                    ..
lindbergh farias     1
linda-gail bekker    1
linda white          1
linda walker         1
șerban nicolae       1
Name: alias, Length: 14593, dtype: int64

In [101]:
data=data[['paragraphs','url']].rename(columns={'paragraphs':'text'})

In [103]:
data.to_json('gu_paragraphs_remove_common_entities_and_downsample_to_3_each.jsonl',lines=True, orient='records')

In [ ]:
##